Import the data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/inaturalist_12K"

Mounted at /content/gdrive
train  val


Install the dependencies

In [ ]:
!pip install wandb

Import the libraries

In [14]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [33]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

Setup to wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

In [16]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable = True)

Preprocess the data

In [18]:
def dataProcess(batch_size = 32, data_aug = True):

    # data augmentation
    if(data_aug):
        train_datagen = ImageDataGenerator(
            rescale = 1./255,
            rotation_range = 90,
            shear_range = 0.2,
            zoom_range = 0.2,
            validation_split = 0.1,
            horizontal_flip = True)
    else:
        train_datagen = ImageDataGenerator( rescale = 1./255, validation_split = 0.1)

    test_datagen = ImageDataGenerator(rescale = 1./255)

    # generate train dataset
    train_generator = train_datagen.flow_from_directory(
        train_path,
        subset='training',
        target_size=(224,224),
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = True,
        seed = 45)

    # generate validation dataset
    val_generator = train_datagen.flow_from_directory(
        train_path,
        subset = 'validation',
        target_size=(224,224),
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = True,
        seed = 45)

    # generate test dataset
    test_generator = test_datagen.flow_from_directory(
        val_path,
        target_size = (224,224),
        batch_size = batch_size,
        class_mode = 'categorical')
    
    return train_generator, val_generator, test_generator

Prepare the pre-trained model for our usage

In [36]:
def buildModel(pre_trained_model = 'InceptionV3'):
    
    # re-size the images as per the imagenet's image size
    IMAGE_SIZE = [224, 224]

    # prepare the pre-trained model excluding the last layer
    if pre_trained_model == 'InceptionV3':
        pre_model = keras.applications.InceptionV3(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)
    if pre_trained_model == 'InceptionResNetV2':
        pre_model = keras.applications.InceptionResNetV2(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)
    if pre_trained_model == 'ResNet50':
        pre_model = keras.applications.ResNet50(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)
    if pre_trained_model == 'Xception':
        pre_model = keras.applications.Xception(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)
    

    # keep the trained weights of the pre trained model same
    for layer in pre_model.layers:
        layer.trainable = False
    
    import os
    numberOfClasses = len(next(os.walk(train_path))[1])

    # flatten the last layer
    x = Flatten()(pre_model.output)
    # add output layer
    prediction = Dense(numberOfClasses, activation='softmax')(x)

    # create final model
    model = Model(inputs = pre_model.input, outputs = prediction)

    model.compile(optimizer = 'adam', 
                  loss = 'categorical_crossentropy', 
                  metrics = ['accuracy']
                  )
    return model

Sweep configurations

In [20]:
sweep_config = {
    'method' : 'random', 
    'metric' : {
        'name': 'val_accuracy',
        'goal': 'maximize'   
        },
    'parameters' : {
        'epochs': {'values' : [5,10]},
        'model' : {'values' : ['InceptionV3','InceptionResNetV2','ResNet50','Xception']},
        'batch_norm' : {'values' : [True,False]},
        'batch_size' : {'values' : [32,64,128]},
        'lr' : {'values' : [1e-4,1e-5]}
    }
}

Train the model

In [37]:
def train():
    config_defaults = {
        "epochs" : 5,
        "model" : 'InceptionV3',
        "batch_norm" : True,
        "batch_size" : 32,
        "lr" : 0.001
    }

    wandb.init(config=config_defaults,name = "CS6910-A2-P_A")
    config = wandb.config
    wandb.run.name = "epochs_{}_model_{}_bn_{}_bs_{}_lr_{}".format(config.epochs,\
                                                      config.model,\
                                                      config.batch_norm,\
                                                      config.batch_size,\
                                                      config.lr)
    wandb.run.save()

    # build model 
    adam = tf.keras.optimizers.Adam(learning_rate = config.lr)
    model = buildModel(config.model)

    # train the model
    trained_model = model.fit(train_set,
                              steps_per_epoch = train_set.samples // config.batch_size,
                              validation_data = val_set, 
                              validation_steps = val_set.samples // config.batch_size,
                              epochs = config.epochs, 
                              callbacks=[WandbCallback(monitor='val_accuracy',mode='max')]
                            )
    
    # evaluate the model
    model.evaluate(test_set,
                   batch_size = config.batch_size,
                   callbacks=[WandbCallback()]
                  )

    wandb.finish()
    return model, trained_model

In [22]:
train_path = '/content/gdrive/My Drive/inaturalist_12K/train'
val_path = '/content/gdrive/My Drive/inaturalist_12K/val'

# process the data
train_set, val_set, test_set = dataProcess(64, True)

Found 9018 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [23]:
sweep_id = wandb.sweep(sweep_config,project = 'cs6910-As-2')

Create sweep with ID: jwbmbg7x
Sweep URL: https://wandb.ai/pandp/cs6910-As-2/sweeps/jwbmbg7x


In [ ]:
wandb.agent(sweep_id, train, count=40)

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 1wfixlzo with config:
wandb: 	batch_norm: True
wandb: 	batch_size: 128
wandb: 	epochs: 5
wandb: 	lr: 0.0001
wandb: 	model: InceptionResNetV2


Epoch 1/5
70/70 [==============================] - 164s 2s/step - loss: 6.3381 - accuracy: 0.5425 - val_loss: 3.7885 - val_accuracy: 0.6228 - _timestamp: 1648583727.0000 - _runtime: 174.0000
Epoch 2/5
70/70 [==============================] - 148s 2s/step - loss: 3.2448 - accuracy: 0.6426 - val_loss: 3.2077 - val_accuracy: 0.6406 - _timestamp: 1648583875.0000 - _runtime: 322.0000
Epoch 3/5
70/70 [==============================] - 151s 2s/step - loss: 3.2563 - accuracy: 0.6402 - val_loss: 3.9202 - val_accuracy: 0.5915 - _timestamp: 1648584028.0000 - _runtime: 475.0000
Epoch 4/5
70/70 [==============================] - 164s 2s/step - loss: 3.5645 - accuracy: 0.6475 - val_loss: 3.6121 - val_accuracy: 0.6362 - _timestamp: 1648584243.0000 - _runtime: 690.0000
Epoch 5/5
32/32 [==============================] - 41s 1s/step - loss: 2.4362 - accuracy: 0.7275



accuracy,▁▇▇▇█
epoch,▁▃▅▆█
loss,█▁▁▂▁
val_accuracy,▄▆▁▅█
val_loss,▇▁█▅▃
accuracy,0.65938
best_epoch,4
best_val_accuracy,0.66295
epoch,4
loss,3.26748
val_accuracy,0.66295


wandb: Agent Starting Run: 43hzryls with config:
wandb: 	batch_norm: False
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	lr: 1e-05
wandb: 	model: InceptionV3


87924736/87910968 [==============================] - 1s 0us/step
Epoch 1/10
140/140 [==============================] - 286s 2s/step - loss: 6.3859 - accuracy: 0.5173 - val_loss: 3.8128 - val_accuracy: 0.5688 - _timestamp: 1648584784.0000 - _runtime: 299.0000
Epoch 2/10
140/140 [==============================] - 264s 2s/step - loss: 3.4312 - accuracy: 0.6042 - val_loss: 4.4563 - val_accuracy: 0.5906 - _timestamp: 1648585089.0000 - _runtime: 604.0000
Epoch 3/10
140/140 [==============================] - 258s 2s/step - loss: 3.5716 - accuracy: 0.6280 - val_loss: 5.0727 - val_accuracy: 0.5865 - _timestamp: 1648585348.0000 - _runtime: 863.0000
Epoch 4/10
140/140 [==============================] - 253s 2s/step - loss: 3.7088 - accuracy: 0.6387 - val_loss: 4.3745 - val_accuracy: 0.6240 - _timestamp: 1648585600.0000 - _runtime: 1115.0000
Epoch 5/10
140/140 [==============================] - 252s 2s/step - loss: 4.1143 - accuracy: 0.6416 - val_loss: 4.3744 - val_accuracy: 0.6302 - _timestamp: 1